In [1]:
import numpy as np
import pandas as pd 

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-talk')
plt.style.use('bmh')
plt.rcParams['font.weight'] = 'medium'
#plt.rcParams['figure.figsize'] = 10,7
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

import os
print(f"Current dir: {os.getcwd()}")
os.chdir('..')
print(f"Current dir: {os.getcwd()}")

/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Current dir: /Users/fernando/Google Drive/Documentos/WORK/financial/adv_financial_ml_book/notebooks
Current dir: /Users/fernando/Google Drive/Documentos/WORK/financial/adv_financial_ml_book


# Chapter 3: Labeling
___

## Exercises
**3.1** Form dollar bars for E-mini S&P 500 futures:
 - **(a)** Apply a symmetric CUSUM filter (Chapter 2, Section 2.5.2.1) where the threshold is the standard deviation of daily returns (Snippet 3.1).
 - **(b)** Use Snippet 3.4 on a pandas series t1, where numDays=1.
 - **(c)** On those sampled features, apply the triple-barrier method, where ptSl=[1,1] and t1 is the series you created in point 1.b.
 - **(d)** Apply getBins to generate the labels.

**3.2** From exercise 1, use Snippet 3.8 to drop rare labels.

**3.3** Adjust the getBins function (Snippet 3.5) to return a 0 whenever the vertical barrier is the one touched first.

**3.4** Develop a trend-following strategy based on a popular technical analysis statistic (e.g., crossing moving averages). For each observation, the model suggests a side, but not a size of the bet.
 - **(a)** Derive meta-labels for ptSl=[1,2] and t1 where numDays=1. Use as trgt the daily standard deviation as computed by Snippet 3.1.
 - **(b)** Train a random forest to decide whether to trade or not. Note: The decision is whether to trade or not, {0,1}, since the underlying model (the crossing moving average) has decided the side, {−1,1}.

**3.5** Develop a mean-reverting strategy based on Bollinger bands. For each observation, the model suggests a side, but not a size of the bet.
 - **(a)** Derive meta-labels for ptSl=[0,2] and t1 where numDays=1. Use as trgt the daily standard deviation as computed by Snippet 3.1.
 - **(b)** Train a random forest to decide whether to trade or not. Use as features: volatility, serial correlation, and the crossing moving averages from exercise 2.
 - **(c)** What is the accuracy of predictions from the primary model (i.e., if the secondary model does not filter the bets)? What are the precision, recall, and F1-scores?
 - **(d)** What is the accuracy of predictions from the secondary model? What are the precision, recall, and F1-scores?

___

### Data loading

In [7]:
clean_data_path = 'data/processed/clean_IVE_tickbidask.parq'
df = pd.read_parquet(clean_data_path)

___
## Solutions

**3.1** Form dollar bars for E-mini S&P 500 futures:
- **(a)** Apply a symmetric CUSUM filter (Chapter 2, Section 2.5.2.1) where the threshold is the standard deviation of daily returns (Snippet 3.1).
- **(b)** Use Snippet 3.4 on a pandas series t1, where numDays=1.
- **(c)** On those sampled features, apply the triple-barrier method, where ptSl=[1,1] and t1 is the series you created in point 1.b.
- **(d)** Apply getBins to generate the labels.

In [ ]:
## form dolar bars 
from src.features.bars_computation import dollar_bar_df
dbars = dollar_bar_df(df, 'dv', 1_000_000).drop_duplicates().dropna()

In [ ]:
price_one_day = df.price['2009-09-28']
dolar_bar_one_day = dbars.price['2009-09-28']

plt.figure(figsize=(20,10))
plt.plot(price_one_day,marker="*")
plt.plot(dolar_bar_one_day, 's', marker="^")

In [27]:
from src.snippets.ch3 import getDailyVol

In [34]:
dbars.head()

,price,bid,ask,size,v,dv
dates,,,,,,
2009-09-28 09:46:35,51.07,51.05,51.07,900,900,45963.0
2009-09-28 09:53:49,51.14,51.13,51.14,2000,2000,102280.0
2009-09-28 09:55:26,51.14,51.11,51.14,100,100,5114.0
2009-09-28 10:02:52,51.25,51.24,51.26,4300,4300,220375.0
2009-09-28 10:10:21,51.29,51.28,51.29,4500,4500,230805.0


In [82]:
close = dbars.price.copy()

In [90]:
close = close.drop_duplicates()
close = close[~close.index.duplicated(keep='first')]

**3.2** From exercise 1, use Snippet 3.8 to drop rare labels.

**3.3** Adjust the getBins function (Snippet 3.5) to return a 0 whenever the vertical barrier is the one touched first.

**3.4** Develop a trend-following strategy based on a popular technical analysis statistic (e.g., crossing moving averages). For each observation, the model suggests a side, but not a size of the bet.
- **(a)** Derive meta-labels for ptSl=[1,2] and t1 where numDays=1. Use as trgt the daily standard deviation as computed by Snippet 3.1.
- **(b)** Train a random forest to decide whether to trade or not. Note: The decision is whether to trade or not, {0,1}, since the underlying model (the crossing moving average) has decided the side, {−1,1}.

**3.5** Develop a mean-reverting strategy based on Bollinger bands. For each observation, the model suggests a side, but not a size of the bet.
- **(a)** Derive meta-labels for ptSl=[0,2] and t1 where numDays=1. Use as trgt the daily standard deviation as computed by Snippet 3.1.
- **(b)** Train a random forest to decide whether to trade or not. Use as features: volatility, serial correlation, and the crossing moving averages from exercise 2.
- **(c)** What is the accuracy of predictions from the primary model (i.e., if the secondary model does not filter the bets)? What are the precision, recall, and F1-scores?
- **(d)** What is the accuracy of predictions from the secondary model? What are the precision, recall, and F1-scores?